In [37]:
import pandas as pd
import numpy as np

# Call individual scripts
final_hcris_v1996 = pd.read_csv('../../data/output/HCRIS_v1996.csv')
final_hcris_v2010 = pd.read_csv('../../data/output/HCRIS_v2010.csv')

# Create missing variables for columns introduced in v2010
final_hcris_v1996['hvbp_payment'] = np.nan
final_hcris_v1996['hrrp_payment'] = np.nan


# Combine v1996 and v2010 datasets
final_hcris = pd.concat([final_hcris_v1996, final_hcris_v2010])

# Convert date columns to datetime format
for col in ['fy_end', 'fy_start', 'date_processed', 'date_created']:
    final_hcris[col] = pd.to_datetime(final_hcris[col], format='%m/%d/%Y')

# Convert to absolute values
final_hcris['tot_discounts'] = final_hcris['tot_discounts'].abs()
final_hcris['hrrp_payment'] = final_hcris['hrrp_payment'].abs()

# Extract fiscal year and sort
final_hcris['fyear'] = final_hcris['fy_end'].dt.year
final_hcris = final_hcris.sort_values(by=['provider_number', 'fyear']).drop(columns=['year'], errors='ignore')

# Count hospitals per year
hospital_counts = final_hcris.groupby('fyear').size()

# Clean data
# Create count of reports by hospital fiscal year
final_hcris['total_reports'] = final_hcris.groupby(['provider_number', 'fyear'])['provider_number'].transform('count')
final_hcris['report_number'] = final_hcris.groupby(['provider_number', 'fyear']).cumcount() + 1

final_hcris.to_csv('../../data/output/HCRIS.csv', index=False)


In [38]:
print(final_hcris)
final_hcris.to_csv('../../data/output/HCRIS.csv', index=False)



      report  provider_number  npi   fy_start     fy_end date_processed  \
242   269995            10001  NaN 2009-10-01 2010-09-30     2011-08-09   
2440  629478            10001  NaN 2016-10-01 2017-09-30     2018-07-19   
24    263654            10005  NaN 2009-10-01 2010-09-30     2011-03-09   
347   620725            10005  NaN 2016-10-01 2017-09-30     2018-03-19   
4767  639416            10006  NaN 2017-07-01 2018-06-30     2018-12-20   
...      ...              ...  ...        ...        ...            ...   
1566  627980           673060  NaN 2017-01-01 2017-12-31     2018-06-22   
3405  634225           673061  NaN 2017-06-01 2018-05-31     2018-11-02   
2587  629813           673062  NaN 2017-01-01 2017-12-31     2018-07-19   
5065  640155           673063  NaN 2017-08-01 2018-07-31     2019-01-24   
3052  632312           673064  NaN 2017-04-01 2018-03-31     2018-09-17   

     date_created  status   beds   tot_charges  ...  \
242    2011-08-05       1  408.0  1.116894e+